In [ ]:
import pennylane as qml                  # Quantum circuit builder and device management
from pennylane import numpy as np        # Not regular NumPy, but similar and supports automatic differentiation
from pennylane import qchem              # Quantum chemistry module used to define molecule Hamiltonians
from pennylane.qchem import excitations  # Single and double excitations used in the UCCSD (Unitary Coupled Cluster Singles and Doubles) ansatz
import matplotlib.pyplot as plt
import os

In [ ]:
"""
Implementation of the VQE algorithm to find the optimum bond-angle of water.
"""

In [ ]:
def water_geometry(angle_deg, bond_length=0.9584):
    # Generate coordinates for water with given angle in degrees
    angle_rad = np.deg2rad(angle_deg)  # Convert to radians
    x = bond_length * np.sin(angle_rad / 2)
    z = bond_length * np.cos(angle_rad / 2)
    
    coordinates = np.array([
        [0.0, 0.0, 0.0],  # Oxygen
        [x, 0.0, z],      # Hydrogen 1
        [-x, 0.0, z]      # Hydrogen 2
    ])
    return coordinates

In [ ]:
def run_vqe(angle_deg, max_iterations=10, stepsize=0.2):
    # Define the molecular Hamiltonian in the default STO-3G basis
    coordinates = water_geometry(angle_deg)
    symbols = ["O", "H", "H"]
    hamiltonian, qubits = qchem.molecular_hamiltonian(symbols, coordinates)

    # Define the Hartree-Fock state
    electrons = 10
    hf = qchem.hf_state(electrons, qubits)

    # # Generate excitation indices
    singles, doubles = excitations(electrons, qubits)

    # Set up a statevector simulator with the number of qubits equal to the number of spin orbitals
    dev = qml.device("default.qubit", wires=qubits)
    

    def ansatz(params_s, params_d):
        # UCCSD-style ansatz
        qml.BasisState(hf, wires=range(qubits))

        # Apply single excitations
        for i, ex in enumerate(singles):
            qml.SingleExcitation(params_s[i], wires=ex)

        # Apply double excitations
        for i, ex in enumerate(doubles):
            qml.DoubleExcitation(params_d[i], wires=ex)


    # Define the VQE cost function
    @qml.qnode(dev)
    def cost_fn(params_s, params_d):
        ansatz(params_s, params_d)

        # Measure the expectation value of the Hamiltonian after applying the ansatz:
        # E(theta) = ⟨ψ(theta)| H |ψ(theta)⟩
        return qml.expval(hamiltonian)
    
    # Adaptive Moment Estimation (Adam) optimizer
    opt = qml.AdamOptimizer(stepsize=stepsize)

    # Initialize zer-vectors as the starting parameters for the ansatz
    theta_s = np.zeros(len(singles), requires_grad=True)
    theta_d = np.zeros(len(doubles), requires_grad=True)

    for n in range(max_iterations):
        # Updates parameters theta to minimize the cost function (energy)
        (theta_s, theta_d), n = opt.step_and_cost(cost_fn, theta_s, theta_d)

    # Return the final optimized energy
    final_energy = cost_fn(theta_s, theta_d)
    return final_energy

In [ ]:
# Defin the angle range and initialize the energy
angles = np.linspace(100, 109, 5)  # Bond angles (in degrees) to test
energies = []

for angle in angles:
    # Iterates over each bond angle
    print(f"Running VQE for angle {angle:.1f}°")

    # Find the optimized ground state energy for that geometry
    energy = run_vqe(angle)
    energies.append(energy)

# Optimized energy plots
plt.plot(angles, energies, marker='o')
plt.xlabel('H–O–H Angle (degrees)')
plt.ylabel('Ground State Energy (Ha)')
plt.title('Water Ground State Energy vs. Bond Angle')
plt.grid(True)
plt.tight_layout()

# Ensure the "images" directory exists
os.makedirs('images', exist_ok=True)

# Save the figure to the "images" directory
plt.savefig('images/Water_Optimal_Angle.png')
plt.show()

# Print the optimum bond-angle and the corresponding energy
min_energy = min(energies)
opt_angle = angles[np.argmin(energies)]
print(f"Minimum energy: {min_energy:.6f} Ha")
print(f"Optimal angle: {opt_angle:.2f}°")